# 学習と検証
p2.xlargeで12時間


## 目標
1. PSPNetの学習と検証の実装
2. セマンティックセグメンテーションのファインチューニングを理解

## Library

In [4]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

In [5]:
# 初期設定
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## DataLoader作成

In [6]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

# ファイルパスリスト作成
# ２章で使ったディレクトリにアクセスする
rootpath = "../2_objectdetection/data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath=rootpath)

# Dataset作成
# (RGB)の色の平均値と標準偏差
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

# DataLoader作成
batch_size = 8

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


## ネットワークモデル作成

In [8]:
from utils.pspnet import PSPNet

# ADE20Kでは１５０クラス分類
# モデルの外側を作ってADE20Kの重みをダウンロードして最後のclassificatonの層を付け替える
# 付け替えた層の重みをxavierの初期値を使って初期化  今回はクラス分類なのでReLUではなくシグモイドを活性化関数に使うから
# ファインチューニングする

net = PSPNet(n_classes=150)
state_dict = torch.load('./weights/pspnet50_ADE20K.pth')   # aws上にダウンロードしてあれば良い

n_classes = 21
net.decode_feature.classification = nn.Conv2d(in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)
net.aux.classification = nn.Conv2d(in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)
            
net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)

print('ネットワーク設定完了：学習済みモデルの重みをロードしました') 

    

ネットワーク設定完了：学習済みモデルの重みをロードしました


In [9]:
net

PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNec

## 損失関数定義

In [10]:
class PSPLoss(nn.Module):
    """
    PSPNetの損失関数クラス
    """
    
    def __init__(self, aux_weight=0.4):
        super(PSPLoss, self).__init__()
        self.aux_weight = aux_weight
        
    def forward(self, outputs, targets):
        """
        損失関数の計算
        
        Parameters
        ----------------
        outputs : PSPNetの出力(tuple)
            (output=torch.Size([num_batch, 21, 475, 475]), output_aux=torch.Size([num_batch, 21, 475, 475]))。
        
        targets : [num_batch, 475, 475]
            正解のアノテーション情報
            
        Returns
        ----------------
        loss : テンソル
            損失の値（普通のlossとauxのlossを足したもの）
            
        """
        
        loss = F.cross_entropy(outputs[0], targets, reduction='mean')
        aux_loss = F.cross_entropy(outputs[1], targets, reduction='mean')
        
        return loss + self.aux_weight*aux_loss
    
criterion = PSPLoss(aux_weight=0.4)
        
        

## 最適化手法定義

In [11]:
# ファインチューニングなので学習率は小さめに設定しておく
optimizer = optim.SGD([
    {'params': net.feature_conv.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_2.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_2.parameters(), 'lr': 1e-3},
    {'params': net.pyramid_pooling.parameters(), 'lr': 1e-3},
    {'params': net.decode_feature.parameters(), 'lr': 1e-2},
    {'params': net.aux.parameters(), 'lr': 1e-2},
], momentum=0.9, weight_decay=0.0001)

# スケジューラの設定
 # 今回はepochごとに学習率を小さくしていく
def lambda_epoch(epoch):
    max_epoch=30
    return math.pow((1-epoch/max_epoch), 0.9)

scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)

## 学習・検証

In [15]:
# モデルを学習させる関数
def train_model(net, detaloaders_dict, criterion, scheduler, optimizer, num_epoch):
    
    # gpu or cpu
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # 高速化させる
    torch.backends.cudnn.benchmark = True
    
    # 画像の枚数
    num_train_imgs = len(dataloaders_dict['train'].dataset)
    num_val_imgs = len(dataloaders_dict['val'].dataset)
    batch_size = dataloaders_dict['train'].batch_size
    
    iteration = 1
    logs = []
    batch_multiplier = 3
    
    # epochのループ
    for epoch in range(num_epochs):
        
        # 開始時刻の保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0
        epoch_val_loss = 0.0
        
        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')
        
        # trainとvalのループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
                scheduler.step()  # 最適化schedulerの更新
                optimizer.zero_grad()
                print('(train)')
                
            else:
                # 検証は５回に１回だけ行う
                if ((epoch+1)%5 == 0):
                    net.eval()
                    print('-------------')
                    print('(val)')
                else:
                    continue
            
            # minibatchを取り出すループ
            count = 0
            for images, anno_class_images, in dataloaders_dict[phase]:
                if images.size()[0] == 1:
                    continue
                    
                # gpuにデータを送る
                images = images.to(device)
                anno_class_images = anno_class_images.to(device)
                
                # multiple minibatchでのパラメータ更新
                if (phase == 'train') and (count==0):
                    optimizer.step()
                    optimizer.zero_grad()
                    count = batch_multiplier  # 3
                    
                # forwardの計算
                with torch.set_grad_enabled(phase=='train'):
                    outputs = net(images)
                    loss = criterion(outputs, anno_class_images.long()) / batch_multiplier
                    
                    # 訓練時はbackprop
                    if phase == 'train':
                        loss.backward()
                        count -= 1
                        
                        if iteration%10 == 0:
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション{} || Loss: {:.4f} || 10iter: {:.4f sec}'.format(iteration, loss.item()/batch_size*batch_multiplier, duration))
                            t_iter_start = time.time()
                            
                        epoch_train_loss += loss.item()*batch_multiplier
                        iteration += 1
                        
                    # 検証時はloss加算するだけ
                    epoch_val_loss += loss.item() * batch_multiplier
                    
        # epochのphaseごとのlossと時間
        print('------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} || Epoch_VAL_Loss: {:.4f}'.format(epoch+1, epoch_train_loss/num_train_imgs, epoch_val_loss/num_val_imgs))
        print('timer: {:.4f} sec'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()
        
        # ログの保存
        log_epoch = {'epoch': epoch+1, 
                     'train_loss': epoch_train_loss/num_train_imgs,
                    'val_loss': epoch_val_loss/num_val_imgs}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv('log_output.csv')
        
    # 最後のネットワークを保存する
    torch.save(net.state_dict(), 'weights/pspnet50_'+str(epoch+1)+'.pth')

In [ ]:
# 学習・検証の実施
num_epochs = 30
train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs=num_epochs)